In [ ]:
def run(PROB, NAME, SEED):
    import socket
    import spynnaker8 as p
    from pyNN.random import NumpyRNG, RandomDistribution
    from pyNN.utility import Timer
    from pyNN.utility.plotting import Figure, Panel
    import matplotlib.pyplot as plt
    import numpy as np

    simulator_name = 'spiNNaker'
    benchmark = 'COBA'

    # exec("from pyNN.%s import *" % simulator_name)

    timer = Timer()

    # === Define parameters ===
    threads = 1
    rngseed = 98766987
    parallel_safe = True

    n = 1500          # number of cells
    r_ei = 4.0        # number of excitatory cells:number of inhibitory cells
    pconn = 0.02      # connection probability
    stim_dur = 100.    # (ms) duration of random stimulation
    rate = 50.       # (Hz) frequency of the random stimulation

    dt = 1.0          # (ms) simulation timestep
    tstop = 1000      # (ms) simulaton duration
    delay = 2

    # Cell parameters
    area = 20000.     # (µm²)
    tau_m = 20.       # (ms)
    cm = 1.           # (µF/cm²)
    g_leak = 5e-5     # (S/cm²)

    E_leak = -60.  # (mV)
    v_thresh = -50.   # (mV)
    v_reset = -60.    # (mV)
    t_refrac = 5.     # (ms) (clamped at v_reset)
    v_mean = -60.     # (mV) mean membrane potential, for calculating CUBA weights
    tau_exc = 5.      # (ms)
    tau_inh = 10.     # (ms)

    # Synapse parameters

    Gexc = 4.     # (nS)
    Ginh = 51.    # (nS)
    Erev_exc = 0.     # (mV)
    Erev_inh = -80.   # (mV)

    # === Calculate derived parameters ===
    area = area * 1e-8                     # convert to cm²
    cm = cm * area * 1000                  # convert to nF
    Rm = 1e-6 / (g_leak * area)            # membrane resistance in MΩ
    assert tau_m == cm * Rm                # just to check

    n_exc = int(round((n * r_ei / (1 + r_ei))))  # number of excitatory cells
    n_inh = n - n_exc                            # number of inhibitory cells

    print("{} {}".format(n_exc, n_inh))


    celltype = p.IF_cond_exp
    w_exc = Gexc * 1e-3              # We convert conductances to uS
    w_inh = Ginh * 1e-3
    print("{} {}".format(w_exc, w_inh))

    # === Build the network ===

    extra = {'threads': threads,
             'filename': "va_%s.xml" % benchmark,
             'label': 'VA'}
    if simulator_name == "neuroml":
        extra["file"] = "VAbenchmarks.xml"

    node_id = p.setup(
        timestep=dt, min_delay=delay, max_delay=delay,
        db_name='va_benchmark.sqlite', **extra)

    cell_params = {'tau_m': tau_m,
                   'tau_syn_E': tau_exc,
                   'tau_syn_I': tau_inh,
                   'v_rest': E_leak,
                   'v_reset': v_reset,
                   'v_thresh': v_thresh,
                   'cm': cm,
                   'tau_refrac': t_refrac,
                   'i_offset': 0
                   }

    print(cell_params)

    if (benchmark == "COBA"):
        cell_params['e_rev_E'] = Erev_exc
        cell_params['e_rev_I'] = Erev_inh

    timer.start()

    print("%s Creating cell populations..." % node_id)
    exc_cells = p.Population(
        n_exc, celltype(**cell_params), label="Excitatory_Cells")
    inh_cells = p.Population(
        n_inh, celltype(**cell_params), label="Inhibitory_Cells")

    print("%s Initialising membrane potential to random values..." % node_id)
    rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
    uniformDistr = RandomDistribution('uniform', [v_reset, v_thresh], rng=rng)
    exc_cells.initialize(v=uniformDistr)
    inh_cells.initialize(v=uniformDistr)

    ###############################################################################
    # thalamic stimulus
    ext_stim = p.Population(
        20, p.SpikeSourcePoisson(rate=rate, duration=stim_dur),
        label="expoisson")
    rconn = 0.01
    ext_conn = p.FixedProbabilityConnector(rconn)

    connections = {}
    connections['ext2e'] = p.Projection(
        ext_stim, exc_cells, ext_conn, receptor_type='excitatory',
        synapse_type=p.StaticSynapse(weight=0.1))
    connections['ext2i'] = p.Projection(
        ext_stim, inh_cells, ext_conn, receptor_type='excitatory',
        synapse_type=p.StaticSynapse(weight=0.1))

    ###############################################################################
    min_delay = 1
    max_delay = min_delay*144
    d0 = 0.04
    v = 0.5
    distance = []
    delay = []
    weight_exc = w_exc
    weight_inh = w_inh
    conn_list_exc = []
    conn_list_inh = []
    
    np.random.seed(SEED)
    for i in range(1500):
        for j in range(1500):
             prob = np.random.uniform(0,1)
             if prob <= np.random.normal(loc=PROB, scale=0.000):
                distance_ = np.linalg.norm(i-j)
                delay_ = 2 #min_delay + distance_ * d0 / v
                #delay_ = np.maximum(delay_, min_delay)
                #delay_ = np.minimum(delay_, max_delay-min_delay) # 
                conn_list_exc += [[i, j, weight_exc, delay_]]
                conn_list_inh += [[i, j, weight_inh, delay_]]

    connections['e2e'] = p.Projection(exc_cells, 
                                exc_cells,
                                connector = p.FromListConnector(conn_list_exc),
                                synapse_type = None,
                                receptor_type = 'excitatory',
                                label = 'e2e'
                                )

    connections['e2i'] = p.Projection(exc_cells, 
                                 inh_cells,
                                connector = p.FromListConnector(conn_list_exc),
                                synapse_type = None,
                                receptor_type = 'excitatory',
                                label = 'e2i'
                                )

    connections['i2e'] = p.Projection(inh_cells, 
                                 exc_cells,
                                connector = p.FromListConnector(conn_list_inh),
                                synapse_type = None,
                                receptor_type = 'inhibitory',
                                label = 'i2e'
                                )

    connections['i2i'] = p.Projection(inh_cells, 
                                 inh_cells,
                                connector = p.FromListConnector(conn_list_inh),
                                synapse_type = None,
                                receptor_type = 'inhibitory',
                                label = 'i2i'
                                )
    #####################################################################################
    # === Setup recording ===
    print("%s Setting up recording..." % node_id)
    exc_cells.record(["spikes", 'v', 'gsyn_inh', 'gsyn_exc'])
    inh_cells.record(["spikes", 'v', 'gsyn_inh', 'gsyn_exc'])
    ext_stim.record("spikes")


    buildCPUTime = timer.diff()

    # === Run simulation ===
    print("%d Running simulation..." % node_id)

    print("timings: number of neurons: {}".format(n))
    print("timings: number of synapses: {}".format(n * n * pconn))

    p.run(tstop)

    simCPUTime = timer.diff()

    # === Print results to file ===

    exc_obs = exc_cells.get_data(['spikes', 'v', 'gsyn_inh', 'gsyn_exc'])
    inh_obs = inh_cells.get_data(['spikes', 'v', 'gsyn_inh', 'gsyn_exc'])
    stim_obs = ext_stim.get_data(['spikes'])


    writeCPUTime = timer.diff()
    np = 1
    if node_id == 0:
        print("\n--- Vogels-Abbott Network Simulation ---")
        print("Nodes                  : %d" % np)
        print("Simulation type        : %s" % benchmark)
        print("Number of Neurons      : %d" % n)
        print("Number of Synapses     : %s" % connections)
        print("Excitatory conductance : %g nS" % Gexc)
        print("Inhibitory conductance : %g nS" % Ginh)
        print("Build time             : %g s" % buildCPUTime)
        print("Simulation time        : %g s" % simCPUTime)
        print("Writing time           : %g s" % writeCPUTime)


    # === Finished with simulator ===

    p.end()

    #########################################################################################à
    import matplotlib.colors as colors
    import numpy as np
    fig, ax = plt.subplots(2,3,figsize=(11,5))
    ax = ax.flatten()
    ax[0].matshow(np.asarray(exc_obs.segments[0].analogsignals[0]).T,  norm=colors.PowerNorm(gamma=1))
    ax[1].matshow(np.asarray(exc_obs.segments[0].analogsignals[1]).T,  norm=colors.PowerNorm(gamma=0.25))
    ax[2].matshow(np.asarray(exc_obs.segments[0].analogsignals[2]).T,  norm=colors.PowerNorm(gamma=0.25))
    ax[3].matshow(np.asarray(inh_obs.segments[0].analogsignals[0]).T,  norm=colors.PowerNorm(gamma=1))
    ax[4].matshow(np.asarray(inh_obs.segments[0].analogsignals[1]).T,  norm=colors.PowerNorm(gamma=0.25))
    ax[5].matshow(np.asarray(inh_obs.segments[0].analogsignals[2]).T,  norm=colors.PowerNorm(gamma=0.25))

    # study network oscillations
    plt.savefig(NAME + '.png')

In [ ]:
import numpy as np
probList = np.random.uniform(0.01,0.1, size=100)
probList=np.unique(probList)

for idx, feat in enumerate(probList):
    PROB = round(feat,4)
    NAME = './nodelay/nodelay_sim_' + str(PROB) + '_' + str(idx)
    SEED = idx
    print(SEED, NAME, PROB)
    run(PROB, NAME, SEED)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
probList = np.random.uniform(0.01,0.1, size=200)
plt.hist(probList)

In [ ]:
probList